In [300]:
import sqlite3
import requests
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import os
import requests
import time
from datetime import datetime, timedelta, timezone

In [123]:
conn = sqlite3.connect("Database/news_data.db")

In [303]:
def create_new_news_table(Ticker, conn):
    """ 
    Ticker: str, the name of the table to be created.
    """
    cursor = conn.cursor()

    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {Ticker}_NewsArticles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ticker TEXT NOT NULL,
            title TEXT NOT NULL,
            url TEXT NOT NULL,
            time_published TEXT NOT NULL,
            closest_time_before_published TEXT NOT NULL,
            authors TEXT, -- Stored as JSON string
            summary TEXT,
            banner_image TEXT,
            source TEXT,
            category_within_source TEXT,
            source_domain TEXT,
            topic TEXT, 
            overall_sentiment_score REAL,
            overall_sentiment_label TEXT,
            relevance_score REAL, 
            UNIQUE(ticker, id)
        )
        """)
    conn.commit()

def create_new_price_table(Ticker, conn):
    """ 
    Ticker: str, the name of the table to be created.
    """
    cursor = conn.cursor()
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {Ticker}PriceHistory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        ticker TEXT NOT NULL,
        sector TEXT NOT NULL,
        UNIQUE(ticker, date)
    )
    ''')
    conn.commit()

def close_connection(conn):
    conn.close()

def insert_NewsData(Ticker, data, conn):
    """
    Ticker: str, the name of the table to insert data into.
    data: list of tuples, each tuple contains the data to be inserted.
    """
    data.to_sql(f'{Ticker}_NewsArticles', conn, if_exists='append', index=False)
    conn.commit()

def insert_PriceHistory(Ticker, data, conn):
    """
    Ticker: str, the name of the table to insert data into.
    data: Dataframe
    """

    data.to_sql(f'{Ticker}PriceHistory', conn, if_exists='append', index=False)
    conn.commit()



In [8]:
def get_alpha_vantage_news(apikey, ticker, from_date=None, to_date=None, sort='RELEVANCE', limit=10):
        """
        Fetch news using Alpha Vantage API
        
        Parameters:
        keywords (str): Search keywords or company name
        from_date (str): Start date in format YYYYMMDD
        to_date (str): End date in format YYYYMMDD
        
        Returns:
        dict: News articles and sentiment information
        """

        if not apikey:
            print("Warning: ALPHA_VANTAGE_API_KEY not found in environment variables")
            return {}
        
        url = 'https://www.alphavantage.co/query'
        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': ticker,
            'apikey': apikey,
            'limit': limit,
            'sort': sort
        }
        from_date = datetime.strptime(from_date, '%Y-%m-%d').strftime('%Y%m%dT%H%M')
        to_date = datetime.strptime(to_date, '%Y-%m-%d').strftime('%Y%m%dT%H%M')
        if from_date and to_date:
            params['time_from'] = from_date
            params['time_to'] = to_date
                
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching news: {response.status_code}")
            return {}

In [3]:
import requests
import pandas as pd
import datetime
import time

def fetch_10min_tiingo_data(ticker, api_token, sector):
    """
    Fetches 2 years of 10-minute interval historical data for a given stock from Tiingo.
    Returns a pandas DataFrame.
    """
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=730)
    interval_days = 60  # ~60 days = safe for 10-min intervals

    current = start_date
    all_data = []

    print(f"Fetching 10-min data for {ticker} from {start_date} to {end_date}...")

    while current < end_date:
        chunk_end = min(current + datetime.timedelta(days=interval_days), end_date)

        url = f"https://api.tiingo.com/iex/{ticker}/prices"
        headers = {
            'Content-Type': 'application/json'
        }
        params = {
            'startDate': current.isoformat(),
            'endDate': chunk_end.isoformat(),
            'resampleFreq': '10min',
            'columns': 'open,high,low,close,volume',
            'token': api_token
        }

        print(f"  → Fetching from {current} to {chunk_end}...")
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data)
        else:
            print(f"  ❌ Error {response.status_code}: {response.text}")

        current = chunk_end + datetime.timedelta(days=1)
        time.sleep(1)  # Add delay to avoid hitting rate limits

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df["Ticker"] = ticker
    df["sector"] = sector  # Example sector, replace with actual if needed

    if not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values('date', inplace=True)

    return df


TEST

In [ ]:
create_new_price_table("AAPL", conn)

In [ ]:
# Replace with your Tiingo API key
API_KEY = "619a6ea449a7a5e044a79c9a2293eee49c339034"

# Fetch for Apple
aapl_df = fetch_10min_tiingo_data("AAPL", API_KEY)

# Save to CSV
aapl_df.to_csv("aapl_10min_2yr.csv", index=False)


Fetching 10-min data for AAPL from 2023-04-24 to 2025-04-23...
  → Fetching from 2023-04-24 to 2023-06-23...
  → Fetching from 2023-06-24 to 2023-08-23...
  → Fetching from 2023-08-24 to 2023-10-23...
  → Fetching from 2023-10-24 to 2023-12-23...
  → Fetching from 2023-12-24 to 2024-02-22...
  → Fetching from 2024-02-23 to 2024-04-23...
  → Fetching from 2024-04-24 to 2024-06-23...
  → Fetching from 2024-06-24 to 2024-08-23...
  → Fetching from 2024-08-24 to 2024-10-23...
  → Fetching from 2024-10-24 to 2024-12-23...
  → Fetching from 2024-12-24 to 2025-02-22...
  → Fetching from 2025-02-23 to 2025-04-23...


In [34]:
insert_PriceHistory("AAPL", aapl_df, conn)

In [35]:
close_connection(conn)

ETL Process

Technology
AAPL – Apple Inc.

MSFT – Microsoft Corp.

NVDA – NVIDIA Corp.

🏦 Financials
JPM – JPMorgan Chase & Co.

BAC – Bank of America

GS – Goldman Sachs Group Inc.

🏘️ Real Estate
PLD – Prologis Inc.

O – Realty Income Corp.

SPG – Simon Property Group Inc.

🛢️ Energy
XOM – Exxon Mobil Corp.

CVX – Chevron Corp.

SLB – Schlumberger Limited

🛍️ E-commerce / Retail
AMZN – Amazon.com Inc.

WMT – Walmart Inc.

TGT – Target Corp.

🍔 Foods and Consumables
KO – Coca-Cola Co.

PG – Procter & Gamble Co.

PEP – PepsiCo Inc.

🏥 Healthcare
JNJ – Johnson & Johnson

PFE – Pfizer Inc.

🎬 Entertainment (Optional extras if you want to expand)
NFLX – Netflix Inc.

DIS – The Walt Disney Company

CMCSA – Comcast Corp.

In [113]:
stocks_by_sector = {
    "Technology": ["AAPL", "MSFT", "NVDA"],
    "Financials": ["JPM", "BAC", "GS"],
    "Real Estate": ["PLD", "O", "SPG"],
    "Energy": ["XOM", "CVX", "SLB"],
    "E-commerce / Retail": ["AMZN", "WMT", "TGT"],
    "Foods and Consumables": ["KO", "PG", "PEP"],
    "Healthcare": ["JNJ", "PFE"],
    "Entertainment": ["NFLX", "DIS", "CMCSA"]
}

print(stocks_by_sector)

{'Technology': ['AAPL', 'MSFT', 'NVDA'], 'Financials': ['JPM', 'BAC', 'GS'], 'Real Estate': ['PLD', 'O', 'SPG'], 'Energy': ['XOM', 'CVX', 'SLB'], 'E-commerce / Retail': ['AMZN', 'WMT', 'TGT'], 'Foods and Consumables': ['KO', 'PG', 'PEP'], 'Healthcare': ['JNJ', 'PFE'], 'Entertainment': ['NFLX', 'DIS', 'CMCSA']}


In [16]:
for sector, stocks in stocks_by_sector.items():
    print(f"Sector: {sector}")
    for stock in stocks:
        print(f"  - {stock}")
        create_new_price_table(stock, conn)

Sector: Technology
  - MSFT
  - NVDA
Sector: Financials
  - JPM
  - BAC
  - GS
Sector: Real Estate
  - PLD
  - O
  - SPG
Sector: Energy
  - XOM
  - CVX
  - SLB
Sector: E-commerce / Retail
  - AMZN
  - WMT
  - TGT
Sector: Foods and Consumables
  - KO
  - PG
  - PEP
Sector: Healthcare
  - JNJ
  - PFE
Sector: Entertainment
  - NFLX
  - DIS
  - CMCSA


In [ ]:
create_new_price_table("AAPL", conn)

In [12]:
API_KEY = "619a6ea449a7a5e044a79c9a2293eee49c339034"

for sector, stocks in stocks_by_sector.items():
    print(f"Sector: {sector}")
    for stock in stocks:
        print(f"  Processing stock: {stock}")
        try:
            stock_df = fetch_10min_tiingo_data(stock, API_KEY, sector)
            insert_PriceHistory(stock, stock_df, conn)
        except Exception as e:
            print(f"❌ Error processing {stock}: {e}")
        print(f"  Inserted {len(stock_df)} records for {stock} into the database.")
    print(f"Finished processing sector: {sector}")
    print("Sleeping for 1 hour to avoid rate limits...")
    time.sleep(3600)  # Add delay to avoid hitting rate limits
    print("Resuming...")

print("All sectors processed.")
close_connection(conn)

Sector: Technology
  Processing stock: MSFT
Fetching 10-min data for MSFT from 2023-04-29 to 2025-04-28...
  → Fetching from 2023-04-29 to 2023-06-28...
  → Fetching from 2023-06-29 to 2023-08-28...
  → Fetching from 2023-08-29 to 2023-10-28...
  → Fetching from 2023-10-29 to 2023-12-28...
  → Fetching from 2023-12-29 to 2024-02-27...
  → Fetching from 2024-02-28 to 2024-04-28...
  → Fetching from 2024-04-29 to 2024-06-28...
  → Fetching from 2024-06-29 to 2024-08-28...
  → Fetching from 2024-08-29 to 2024-10-28...
  → Fetching from 2024-10-29 to 2024-12-28...
  → Fetching from 2024-12-29 to 2025-02-27...
  → Fetching from 2025-02-28 to 2025-04-28...
  Inserted 20280 records for MSFT into the database.
  Processing stock: NVDA
Fetching 10-min data for NVDA from 2023-04-29 to 2025-04-28...
  → Fetching from 2023-04-29 to 2023-06-28...
  → Fetching from 2023-06-29 to 2023-08-28...
  → Fetching from 2023-08-29 to 2023-10-28...
  → Fetching from 2023-10-29 to 2023-12-28...
  → Fetching fro

In [43]:
stocks_by_sector["Foods and Consumables"]

['KO', 'PG', 'PEP']

In [ ]:
for sector, stocks in stocks_by_sector.items():
    if sector != "Foods and Consumables":
        continue
    print(f"Sector: {sector}")
    for stock in stocks:
        if stock not in ("PG", "PEP"):
            continue                    # Only continue with the next exe if stock is either PG or PEP
        print(f"  Processing stock: {stock}")

Sector: Foods and Consumables
  Processing stock: PG
  Processing stock: PEP


In [59]:
safty_list = []

for sector, stocks in stocks_by_sector.items():
    if sector != "Foods and Consumables":
        continue
    print(f"Sector: {sector}")
    for stock in stocks:
        if stock not in ("PG", "PEP"):
            continue
        print(f"  Processing stock: {stock}")
        try:
            stock_df = fetch_10min_tiingo_data(stock, API_KEY, sector)
            safty_list.append(stock_df)  # Append the DataFrame to the list
            insert_PriceHistory(stock, stock_df, conn)
        except Exception as e:
            print(f"❌ Error processing {stock}: {e}")
        print(f"  Inserted {len(stock_df)} records for {stock} into the database.")
    print(f"Finished processing sector: {sector}")
print("All sectors processed.")
close_connection(conn)

Sector: Foods and Consumables
  Processing stock: PG
Fetching 10-min data for PG from 2023-04-30 to 2025-04-29...
  → Fetching from 2023-04-30 to 2023-06-29...
  → Fetching from 2023-06-30 to 2023-08-29...
  → Fetching from 2023-08-30 to 2023-10-29...
  → Fetching from 2023-10-30 to 2023-12-29...
  → Fetching from 2023-12-30 to 2024-02-28...
  → Fetching from 2024-02-29 to 2024-04-29...
  → Fetching from 2024-04-30 to 2024-06-29...
  → Fetching from 2024-06-30 to 2024-08-29...
  → Fetching from 2024-08-30 to 2024-10-29...
  → Fetching from 2024-10-30 to 2024-12-29...
  → Fetching from 2024-12-30 to 2025-02-28...
  → Fetching from 2025-03-01 to 2025-04-29...
  Inserted 20291 records for PG into the database.
  Processing stock: PEP
Fetching 10-min data for PEP from 2023-04-30 to 2025-04-29...
  → Fetching from 2023-04-30 to 2023-06-29...
  → Fetching from 2023-06-30 to 2023-08-29...
  → Fetching from 2023-08-30 to 2023-10-29...
  → Fetching from 2023-10-30 to 2023-12-29...
  → Fetching 

Test for news data

In [304]:
for sector, stocks in stocks_by_sector.items():
    print(f"Sector: {sector}")
    for stock in stocks:
        print(f"  Creating news table for: {stock}")
        cursor = conn.cursor()
        cursor.execute(f"DROP TABLE IF EXISTS {stock}_NewsArticles")
        create_new_news_table(stock, conn)
    
print("All news tables created.")
        

Sector: Technology
  Creating news table for: AAPL
  Creating news table for: MSFT
  Creating news table for: NVDA
Sector: Financials
  Creating news table for: JPM
  Creating news table for: BAC
  Creating news table for: GS
Sector: Real Estate
  Creating news table for: PLD
  Creating news table for: O
  Creating news table for: SPG
Sector: Energy
  Creating news table for: XOM
  Creating news table for: CVX
  Creating news table for: SLB
Sector: E-commerce / Retail
  Creating news table for: AMZN
  Creating news table for: WMT
  Creating news table for: TGT
Sector: Foods and Consumables
  Creating news table for: KO
  Creating news table for: PG
  Creating news table for: PEP
Sector: Healthcare
  Creating news table for: JNJ
  Creating news table for: PFE
Sector: Entertainment
  Creating news table for: NFLX
  Creating news table for: DIS
  Creating news table for: CMCSA
All news tables created.


In [76]:
alpha_news = get_alpha_vantage_news(apikey='R6WUON3CB6JRP1K7', ticker='AAPL', from_date='2023-01-01', to_date='2023-01-02', limit=50)

In [77]:
len(alpha_news['feed'])  # Check the length of the feed

3

In [78]:
alpha_news['feed']

[{'title': 'The Year Big Tech Stocks Fell From Glory',
  'url': 'https://www.wsj.com/articles/the-year-big-tech-stocks-fell-from-glory-11672543952',
  'time_published': '20230101T103000',
  'authors': ['Hannah Miao'],
  'summary': 'With interest rates rising sharply, momentum shifted in favor of defensive plays, while the energy sector rallied amid geopolitical strife.',
  'banner_image': 'https://images.wsj.net/im-695150/social',
  'source': 'Wall Street Journal',
  'category_within_source': 'Markets',
  'source_domain': 'www.wsj.com',
  'topics': [{'topic': 'Technology', 'relevance_score': '1.0'},
   {'topic': 'Financial Markets', 'relevance_score': '0.360215'}],
  'overall_sentiment_score': 0.332214,
  'overall_sentiment_label': 'Somewhat-Bullish',
  'ticker_sentiment': [{'ticker': 'NFLX',
    'relevance_score': '0.436009',
    'ticker_sentiment_score': '0.357976',
    'ticker_sentiment_label': 'Bullish'},
   {'ticker': 'GOOG',
    'relevance_score': '0.751437',
    'ticker_sentimen

In [79]:
time_published_list = [item['time_published'] for item in alpha_news["feed"] if 'time_published' in item]
print(time_published_list)

['20230101T103000', '20230101T101300', '20230101T105000']


In [253]:
from collections import defaultdict

# Group timestamps by date
grouped_by_date = defaultdict(list)
for timestamp in time_published_list:
    date = timestamp[:8]  # Extract the date portion (YYYYMMDD)
    grouped_by_date[date].append(timestamp)

# Convert defaultdict to a regular dictionary for better readability
grouped_by_date = dict(grouped_by_date)

print(grouped_by_date)

{'20230101': ['20230101T103000', '20230101T101300', '20230101T105000']}


In [254]:
for name,dict_ in grouped_by_date.items():
    print(f"{name}")

20230101


In [255]:
for name,dict_ in alpha_news["feed"][0].items():
    print ('the name of the dictionary is ', name)
    print ('the dictionary looks like ', dict_)
    print ('the dictionary type is ', type(dict_))

the name of the dictionary is  title
the dictionary looks like  The Year Big Tech Stocks Fell From Glory
the dictionary type is  <class 'str'>
the name of the dictionary is  url
the dictionary looks like  https://www.wsj.com/articles/the-year-big-tech-stocks-fell-from-glory-11672543952
the dictionary type is  <class 'str'>
the name of the dictionary is  time_published
the dictionary looks like  20230101T103000
the dictionary type is  <class 'str'>
the name of the dictionary is  authors
the dictionary looks like  ['Hannah Miao']
the dictionary type is  <class 'list'>
the name of the dictionary is  summary
the dictionary looks like  With interest rates rising sharply, momentum shifted in favor of defensive plays, while the energy sector rallied amid geopolitical strife.
the dictionary type is  <class 'str'>
the name of the dictionary is  banner_image
the dictionary looks like  https://images.wsj.net/im-695150/social
the dictionary type is  <class 'str'>
the name of the dictionary is  sou

In [256]:
alpha_news["feed"]

[{'title': 'The Year Big Tech Stocks Fell From Glory',
  'url': 'https://www.wsj.com/articles/the-year-big-tech-stocks-fell-from-glory-11672543952',
  'time_published': '20230101T103000',
  'authors': ['Hannah Miao'],
  'summary': 'With interest rates rising sharply, momentum shifted in favor of defensive plays, while the energy sector rallied amid geopolitical strife.',
  'banner_image': 'https://images.wsj.net/im-695150/social',
  'source': 'Wall Street Journal',
  'category_within_source': 'Markets',
  'source_domain': 'www.wsj.com',
  'topics': [{'topic': 'Technology', 'relevance_score': '1.0'},
   {'topic': 'Financial Markets', 'relevance_score': '0.360215'}],
  'overall_sentiment_score': 0.332214,
  'overall_sentiment_label': 'Somewhat-Bullish',
  'ticker_sentiment': [{'ticker': 'NFLX',
    'relevance_score': '0.436009',
    'ticker_sentiment_score': '0.357976',
    'ticker_sentiment_label': 'Bullish'},
   {'ticker': 'GOOG',
    'relevance_score': '0.751437',
    'ticker_sentimen

In [288]:
news = pd.DataFrame([alpha_news["feed"][0]])
for i in range(1, len(alpha_news["feed"])):
    news1 = pd.DataFrame([alpha_news["feed"][i]])
    news = pd.concat([news, news1], ignore_index=True)

In [289]:
news

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,The Year Big Tech Stocks Fell From Glory,https://www.wsj.com/articles/the-year-big-tech...,20230101T103000,[Hannah Miao],"With interest rates rising sharply, momentum s...",https://images.wsj.net/im-695150/social,Wall Street Journal,Markets,www.wsj.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.332214,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.4360..."
1,2 Beaten-Down Warren Buffett Stocks to Buy in ...,https://www.fool.com/investing/2023/01/01/beat...,20230101T101300,[Cory Renauer],These are high-conviction stocks for the Oracl...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.271653,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.0565..."
2,80% of Warren Buffett's Portfolio Is Invested ...,https://www.fool.com/investing/2023/01/01/80-o...,20230101T105000,[Keith Speights],Two of these stocks were especially big winner...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.232526,Somewhat-Bullish,"[{'ticker': 'OXY', 'relevance_score': '0.11923..."


In [259]:

for i in range(len(news.iloc[0])):
    print(news.iloc[0][i])
    print(type(news.iloc[0][i]))

    if isinstance(news.iloc[0][i], str):
        print('yes')
    else:
        print('no')
    print('------------------')

The Year Big Tech Stocks Fell From Glory
<class 'str'>
yes
------------------
https://www.wsj.com/articles/the-year-big-tech-stocks-fell-from-glory-11672543952
<class 'str'>
yes
------------------
20230101T103000
<class 'str'>
yes
------------------
['Hannah Miao']
<class 'list'>
no
------------------
With interest rates rising sharply, momentum shifted in favor of defensive plays, while the energy sector rallied amid geopolitical strife.
<class 'str'>
yes
------------------
https://images.wsj.net/im-695150/social
<class 'str'>
yes
------------------
Wall Street Journal
<class 'str'>
yes
------------------
Markets
<class 'str'>
yes
------------------
www.wsj.com
<class 'str'>
yes
------------------
[{'topic': 'Technology', 'relevance_score': '1.0'}, {'topic': 'Financial Markets', 'relevance_score': '0.360215'}]
<class 'list'>
no
------------------
0.332214
<class 'numpy.float64'>
no
------------------
Somewhat-Bullish
<class 'str'>
yes
------------------
[{'ticker': 'NFLX', 'relevance_

C:\Users\andre\AppData\Local\Temp\ipykernel_71116\2626120922.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(news.iloc[0][i])
C:\Users\andre\AppData\Local\Temp\ipykernel_71116\2626120922.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(type(news.iloc[0][i]))
C:\Users\andre\AppData\Local\Temp\ipykernel_71116\2626120922.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(news.iloc[0][i], str):


In [296]:
def get_closest_time_before_published(time_str):
    # Convert string to datetime
    dt = datetime.strptime(time_str, '%Y%m%dT%H%M%S').replace(tzinfo=timezone.utc)

    # Round down to the previous 10-minute mark
    minutes = (dt.minute // 10) * 10
    rounded = dt.replace(minute=minutes, second=0, microsecond=0)

    # Define valid trading window
    start_time = rounded.replace(hour=13, minute=30)
    end_time = rounded.replace(hour=19, minute=50)

    if start_time <= rounded <= end_time:
        return rounded.isoformat(sep=' ')
    elif rounded < start_time:
        # Move to the previous day at 13:30 UTC
        previous_day = (dt - timedelta(days=1)).replace(hour=19, minute=50, second=0, microsecond=0)
        return previous_day.isoformat(sep=' ')
    else:
        # Move to the next day at 13:30 UTC
        next_day = (dt + timedelta(days=1)).replace(hour=13, minute=30, second=0, microsecond=0)
        return next_day.isoformat(sep=' ')


def get_relevance(entry, ticker="AAPL"):
    if isinstance(entry, list):
        for item in entry:
            if item.get("ticker") == ticker:
                return float(item.get("relevance_score", 0))
    return None  # or 0.0 if you prefer

def get_highest_scored_topic(entry):
    if isinstance(entry, list):
        # Sort the list of dictionaries by relevance_score in descending order
        sorted_topics = sorted(entry, key=lambda x: float(x.get('relevance_score', 0)), reverse=True)
        # Return the topic of the highest scored entry
        return sorted_topics[0].get('topic', '') if sorted_topics else None
    return None  # or "" if you'd prefer an empty string


In [291]:
# Apply the function to the 'topics' column
news["topic"] = news["topics"].apply(get_highest_scored_topic)
news["relevance_score"] = news["ticker_sentiment"].apply(lambda x: get_relevance(x, ticker="AAPL"))
news["closest_time_before_published"] = news["time_published"].apply(get_closest_time_before_published)
news['authors'] = news['authors'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else '')
news = news.drop(columns=["topics"])
news = news.drop(columns=["ticker_sentiment"])


In [292]:
# Get the first element in the 'authors' list
news


,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,topic,relevance_score,closest_time_before_published
0,The Year Big Tech Stocks Fell From Glory,https://www.wsj.com/articles/the-year-big-tech...,20230101T103000,Hannah Miao,"With interest rates rising sharply, momentum s...",https://images.wsj.net/im-695150/social,Wall Street Journal,Markets,www.wsj.com,0.332214,Somewhat-Bullish,Technology,0.436009,2022-12-31 19:50:00+00:00
1,2 Beaten-Down Warren Buffett Stocks to Buy in ...,https://www.fool.com/investing/2023/01/01/beat...,20230101T101300,Cory Renauer,These are high-conviction stocks for the Oracl...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,0.271653,Somewhat-Bullish,Earnings,0.277118,2022-12-31 19:50:00+00:00
2,80% of Warren Buffett's Portfolio Is Invested ...,https://www.fool.com/investing/2023/01/01/80-o...,20230101T105000,Keith Speights,Two of these stocks were especially big winner...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,0.232526,Somewhat-Bullish,Financial Markets,0.178021,2022-12-31 19:50:00+00:00


In [266]:
news.iloc[0][9]

C:\Users\andre\AppData\Local\Temp\ipykernel_71116\1537425571.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  news.iloc[0][9]


np.float64(0.332214)

In [311]:
def get_news_data(apikey='R6WUON3CB6JRP1K7', ticker='AAPL', from_date='2023-05-01', to_date='2025-05-01', limit=1000):
    # keys lol: IG5JCBZ47L987AB9, R6WUON3CB6JRP1K7: free to get at alphavantage.co
    start_date = datetime.strptime(from_date, '%Y-%m-%d')
    end_date = datetime.strptime(to_date, '%Y-%m-%d')
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=365)
        try:
            print(f"{current_date.strftime('%Y-%m-%d')} - {next_date.strftime('%Y-%m-%d')}")
            alpha_news = get_alpha_vantage_news(apikey=apikey, ticker=ticker, from_date=current_date.strftime('%Y-%m-%d'), to_date=next_date.strftime('%Y-%m-%d'), limit=limit)
            print(f"{len(alpha_news['feed'])} news articles fetched from {current_date.strftime('%Y-%m-%d')}")
            # Add your logic here, e.g., calling a function with these dates
            news = pd.DataFrame([alpha_news["feed"][0]])
            for i in range(1, len(alpha_news["feed"])):
                news1 = pd.DataFrame([alpha_news["feed"][i]])
                news = pd.concat([news, news1], ignore_index=True)

                # Clean dataframe
            news["topic"] = news["topics"].apply(get_highest_scored_topic)
            news["relevance_score"] = news["ticker_sentiment"].apply(lambda x: get_relevance(x, ticker="AAPL"))
            news["closest_time_before_published"] = news["time_published"].apply(get_closest_time_before_published)
            news['authors'] = news['authors'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else '')
            news = news.drop(columns=["topics"])
            news = news.drop(columns=["ticker_sentiment"])
            news["ticker"] = ticker
            insert_NewsData(ticker, news, conn)
        except Exception as e:
            print(f"❌ Error processing news between {current_date.strftime('%Y-%m-%d')} - {next_date.strftime('%Y-%m-%d')}: {e}")
            print(alpha_news)    

        current_date = next_date

In [ ]:
get_news_data(apikey='R6WUON3CB6JRP1K7', ticker='AAPL', from_date='2023-05-02', to_date='2025-04-29', limit=1000)

2023-05-02 - 2024-05-01
❌ Error processing news between 2023-05-02 - 2024-05-01: 'feed'
{'Information': 'We have detected your API key as WCL5ES1EON307LZA and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
2024-05-01 - 2025-05-01
❌ Error processing news between 2024-05-01 - 2025-05-01: 'feed'
{'Information': 'We have detected your API key as WCL5ES1EON307LZA and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [314]:
close_connection(conn)

In [307]:
alpha_news = get_alpha_vantage_news(apikey="R6WUON3CB6JRP1K7", ticker="AAPL", from_date="2023-05-13", to_date="2023-05-14", limit=50)

In [308]:
alpha_news

{'Information': 'We have detected your API key as R6WUON3CB6JRP1K7 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}